In [65]:
# %pip install \
#     "pinecone" \
#     "langchain-chromadb" \
#     "langchain-openai" \
#     "langchain-text-splitters" \
#     "langchain" \
#     "langchain-embeddings" \
#     "python-frontmatter" \
#     "PyPDF2" \
#     "sklearn"\
#     "langchain_community"\
#     ""\
#     ""\
#     ""\
#     ""\
# %pip install -qU langchain-text-splitters
# %pip install -qU langchain-ollama
    


In [66]:
from langchain.document_loaders import PyPDFLoader
import frontmatter

def pdf_to_md(pdf_path, md_path):
    # Load the PDF
    loader = PyPDFLoader(pdf_path)
    documents = loader.load()
    
    # Extract metadata
    metadata = documents[0].metadata if documents else {}
    
    # Extract content
    content = "\n\n".join(doc.page_content for doc in documents)
    
    # Create frontmatter metadata block
    post = frontmatter.Post(content)
    for key, value in metadata.items():
        post[key] = value
    
    # Write to Markdown file
    with open(md_path, "w", encoding="utf-8") as md_file:
        md_file.write(frontmatter.dumps(post))
    
    print(f"Markdown file created at {md_path}")

# Input and output paths
pdf_path = r"D:\MSAIM\trimister-2 msaiml\Adv-Machine_learning\ChatBot_edt\pdf\The Subtle Art of Not Giving a F_ck-5-99.pdf"  # Replace with your PDF file path
md_path = r"D:\MSAIM\trimister-2 msaiml\Adv-Machine_learning\ChatBot_edt\md\demo.md"    # Replace with desired Markdown file path

pdf_to_md(pdf_path, md_path)


Markdown file created at D:\MSAIM\trimister-2 msaiml\Adv-Machine_learning\ChatBot_edt\md\demo.md


In [67]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
with open(r"D:\MSAIM\trimister-2 msaiml\Adv-Machine_learning\ChatBot_edt\md\demo.md", encoding="utf-8") as f:
    sangf = f.read()


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=500,
    length_function=len,
    add_start_index=True,

)
# chunks = text_splitter.split_documents(sangf)
texts = text_splitter.create_documents([sangf])
# sangf

In [68]:
print(texts[0])
print(texts[1])

page_content='---
page: 0
source: D:\MSAIM\trimister-2 msaiml\Adv-Machine_learning\ChatBot_edt\pdf\The Subtle
  Art of Not Giving a F_ck-5-99.pdf
---' metadata={'start_index': 0}
page_content='CHAPTER 1
Don’t Try
Charles Bukowski was an alcoholic, a womanizer, a chronic
gambler, a lout, a cheapskate, a deadbeat, and on his worst
days, a poet. He’s probably the last person on earth you
would ever look to for life advice or expect to see in any sort
of self-help book.
Which is why he’s the perfect place to start.
Bukowski wanted to be a writer. But for decades his work
was rejected by almost every magazine, newspaper, journal,
agent, and publisher he submitted to. His work was horrible,
they said. Crude. Disgusting. Depraved. And as the stacks of
rejection slips piled up, the weight of his failures pushed him
deep into an alcohol-fueled depression that would follow
him for most of his life.
Bukowski had a day job as a letter-ﬁler at a post oﬃce.
He got paid shit money and spent most of i

In [69]:
import chromadb.utils.embedding_functions as embedding_functions
chunks = text_splitter.split_text(sangf)

embed_model_ollama = embedding_functions.OllamaEmbeddingFunction(
    url="http://localhost:11434/api/embeddings",
    model_name="llama3.2",
)


In [70]:
%pip install torch

  Using cached torch-2.5.1-cp312-cp312-win_amd64.whl.metadata (28 kB)
Using cached torch-2.5.1-cp312-cp312-win_amd64.whl (203.0 MB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ------------------------ --------------- 1.0/1.7 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.3.1 requires Pillow, which is not installed.
sentence-transformers 3.3.1 requires scikit-learn, which is not installed.
sentence-transformers 3.3.1 requires scipy, which is not installed.


In [72]:
from torch import cuda
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'device': device, 'batch_size': 32},
)

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [ ]:
embeddings = embed_model_ollama(chunks)

print(f"We have {len(embeddings)} doc embeddings, each with "
      f"a dimensionality of {len(embeddings[0])}.")

We have 279 doc embeddings, each with a dimensionality of 3072.


In [2]:
from torch import cuda
if cuda.is_available():
    print(f"GPU: {cuda.get_device_name()}")
else:
    print("No GPU available")
    

GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [1]:
import torch
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Should return True if CUDA is enabled
print(torch.version.cuda)  

2.5.1+cu121
True
12.1


In [ ]:
# Generate metadata
metadata = [{"chunk_id": i, "text": chunk} for i, chunk in enumerate(chunks)]

# Prepare data for ChromaDB
chroma_data = [
    {
        "id": f"chunk-{i}",        # Unique ID for the chunk
        "metadata": metadata[i],  # Metadata dictionary
        "embedding": embeddings[i]  # Embedding vector
    }
    for i in range(len(chunks))
]

# Example: Access a single chunk's data
print(chroma_data[1])


{'id': 'chunk-1', 'metadata': {'chunk_id': 1, 'text': 'CHAPTER 1\nDon’t Try\nCharles Bukowski was an alcoholic, a womanizer, a chronic\ngambler, a lout, a cheapskate, a deadbeat, and on his worst\ndays, a poet. He’s probably the last person on earth you\nwould ever look to for life advice or expect to see in any sort\nof self-help book.\nWhich is why he’s the perfect place to start.\nBukowski wanted to be a writer. But for decades his work\nwas rejected by almost every magazine, newspaper, journal,\nagent, and publisher he submitted to. His work was horrible,\nthey said. Crude. Disgusting. Depraved. And as the stacks of\nrejection slips piled up, the weight of his failures pushed him\ndeep into an alcohol-fueled depression that would follow\nhim for most of his life.\nBukowski had a day job as a letter-ﬁler at a post oﬃce.\nHe got paid shit money and spent most of it on booze. He\ngambled away the rest at the racetrack. At night, he would\ndrink alone and sometimes hammer out poetry on

In [ ]:
from chromadb import Client

# Initialize ChromaDB client
client = Client()

# Create a collection
collection = client.get_or_create_collection(name="sangf_rag")

# Upsert data
for data in chroma_data:
    collection.add(
        ids=[data["id"]],
        embeddings=[data["embedding"]],
        metadatas=[data["metadata"]]
    )


Add of existing embedding ID: chunk-0
Insert of existing embedding ID: chunk-0
Add of existing embedding ID: chunk-1
Insert of existing embedding ID: chunk-1
Add of existing embedding ID: chunk-2
Insert of existing embedding ID: chunk-2
Add of existing embedding ID: chunk-3
Insert of existing embedding ID: chunk-3
Add of existing embedding ID: chunk-4
Insert of existing embedding ID: chunk-4
Add of existing embedding ID: chunk-5
Insert of existing embedding ID: chunk-5
Add of existing embedding ID: chunk-6
Insert of existing embedding ID: chunk-6
Add of existing embedding ID: chunk-7
Insert of existing embedding ID: chunk-7
Add of existing embedding ID: chunk-8
Insert of existing embedding ID: chunk-8
Add of existing embedding ID: chunk-9
Insert of existing embedding ID: chunk-9
Add of existing embedding ID: chunk-10
Insert of existing embedding ID: chunk-10
Add of existing embedding ID: chunk-11
Insert of existing embedding ID: chunk-11
Add of existing embedding ID: chunk-12
Insert of

In [ ]:
# Ensure the embedding model used for querying is the same as the one used for creating the collection
query_embedding = embed_model_ollama(["happiness is a problem"])

# Query the collection to retrieve all documents
results = collection.query(
    query_embeddings=query_embedding,  # Use the embedding of the query
    n_results=3  # Number of documents to retrieve
)

print(f"Found {len(results)} results.", results )

Found 8 results. {'ids': [['chunk-180', 'chunk-71', 'chunk-9']], 'embeddings': None, 'documents': [[None, None, None]], 'uris': None, 'data': None, 'metadatas': [[{'chunk_id': 180, 'text': 'be some mutual respect (which there is). Or maybe mutual\ntrust is what to look for (and it’s there). Perhaps these\nmetrics would be better assessments of brotherhood than\nhow many text messages he and I exchange.\nThis clearly makes sense; it feels true for me. But it still\nfucking hurts that my brother and I aren’t close. And there’s\nno positive way to spin it. There’s no secret way to glorify\nmyself through this knowledge. Sometimes brothers—even\nbrothers who love each other—don’t have close\nrelationships, and that’s ﬁne. It is hard to accept at ﬁrst, but\nthat’s ﬁne. What is objectively true about your situation is\nnot as important as how you come to see the situation, how\nyou choose to measure it and value it. Problems may be\ninevitable, but the meaning of each problem is not. We get\

In [ ]:
from langchain.vectorstores import Chroma

# Use the existing client and collection
vectorstore = Chroma(
    client=client,
    collection_name="sangf_rag",
    embedding_function=embed_model_ollama,
    persist_directory="chroma_db",
)


In [ ]:
query = 'Happiness Is a Problem'

# Embed the query
query_embedding = embed_model_ollama([query])

results = vectorstore.similarity_search(
    query_embedding,  # the search query embedding
    k=3  # returns top 3 most relevant chunks of text
)

print(results)

AttributeError: 'OllamaEmbeddingFunction' object has no attribute 'embed_query'